# **INTRODUCTION TO MEDICAL IMAGING IN PYTHON**


# OBJECTIVE

Information from medical images represents one of the important sources of information in the field of Biomedical Data Science. However, the complexity of this type of structured data requires specific tools and procedures in order to maximize the information that can be obtained from these images.

The main objective of this practical session is to become familiar with the handling and processing of medical images. 

The following practical session is divided in three main blocks:
* In the first block we will study how to read different formats of biomedical images as well as how to extract the crucial metadata for their correct interpretation.
* In the second block we will focus on basic image processing operations and the application of different coordinate transformations.
* In the last block we will define and apply a complete processing pipeline for neuroimaging.

# MATERIALS

* In this practical lesson we will use the following images
    * *TCIA-DICOM/1-014.dcm* This DICOM file provide from TCIA-GBM open dataset.
    * *IVY_GAP_W01* This folder contains different MRI image from a single brain tumor study.
    * *MNI_ATLAS* This folder contains the MRI MNI atlas.
* The images needed for this practice are available through the following link.
* In order to work with these images from google colab, you can upload them to your Google Drive space.
* Then just run the following code to mount your google drive and have it accessible from colab. 

```
from google.colab import drive
drive.mount('/content/drive')
```





# METHODOLOGY
# Part 1 - Medical image handling (loading, basic processing and saving)

## 1.1. Introduction 


*   In this first part of the lab we will see how to read DICOM and NIFTI files in python and extract the image information and its contextual metadata. 
*   We will also apply basic operations seen in theory to the images. These include intensity changes, contrast changes, histogram matching, and application of common filters such as gaussian or edge enhancement. 
*   Finally we will save the generated images as nifti for later viewing and comparison.


## 1.2. Reading a medical image in DICOM format

### Import the required libaries

* To be able to read dicom data we need to import "dcread" function from *pydicom* library. 
* Additionally for being able to plot the image data embeded in the dicom file we will import *matplotlib.pytplot*
* Note: to import libraries not pre-included in Google colab you can use the command "! pip install *requered library*".


### Load the dicom file 
Load the dicom file *TCIA-DICOM/1-014.dcm* using the dcmread command

Explore the metadata asociated with the dicom file. In particular obtain the following parameters:


*   Patient ID
*   Modality
*   Study date
*   Image size
*   Pixel Spacing



The image data is located in the *pixel_array* attribute of the imported dicom object.

Now you can plot the image data obtained using imshow from matplotlib.

## 1.3 Reading a medical image in NIFTI format



### Import the required libraries
Import *nibabel* library

### Load a nifti file

* Use the *nibabel.load* comand to load the *IVY_GAP_W01/T1c/T1.nii* nifti image. 
* As a result we will obtain a nibabel image object.

As with any Python object, you can inspect *img* to see what attributes it has. We recommend using IPython tab completion for this, but here are some examples of interesting attributes:


### Exploring the nibabel object attributtes

A nibabel image object is the association of three things:

* an N-D array containing the image data;
* a (4, 4) affine matrix mapping array coordinates to coordinates in some RAS+ world coordinate space (Coordinate systems and affines);
* image metadata in the form of a header

#### Affine matrix

**affine** is the affine array relating array coordinates from the image data array to coordinates in some RAS+ world coordinate system ([Coordinate systems and affines](https://https://nipy.org/nibabel/coordinate_systems.html)):

Print the affine attributte of the nibabel object 

#### Image metadata

The header of an image contains the image metadata. The information in the header will differ between different image formats. For example, the header information for a NIfTI1 format file differs from the header information for a MINC format file.

Our image is a NIfTI1 format image, and it therefore has a NIfTI1 format header:

*Note: For the record, srow_x, srow_y, srow_z is the affine matrix of the image. Bitpix is how many bits we use to represent each pixel intensity.*

The header of any image will normally have the following methods:

* get_data_shape() to get the output shape of the image data array

* get_data_dtype() to get the numpy data type in which the image data is stored (or will be stored if you save the image)

* get_zooms() to get the voxel sizes in millimeters

The last value of header.get_zooms() is the time between scans in milliseconds (for dynamic sequences); this is the equivalent of voxel size on the time axis.

Apply these methods to our loaded image:

#### Image data array
We can get the data from the nifti object with the get_fdata() method.
For the array image, get_fdata() just returns the data array.

We can check the shape of the array provided by image_data.

Now just select a 2D slice in the middle of the field of view (the one on the middle of the z-axis) to plot it with *imshow* command.

## 1.4 Basic processing 

Our goal now will be to be able to modify the image using some of the basic image processing methods we have seen in the theory classes.


### Use of the histogram to analyze changes in intensity distributions

First of all we will analyse the histogram of the image. To do so we can use the *plt.hist* function included in *matplotlib* library. To show the relation between the image and the histogram is convenient to plot both the histogram and an 2D slice (e.g. the one in the middle of the field of view) side by side. 

Note: To see the results of the brightness and contrast increase we need to plot the images with the same intensity range, if not *imshow* command will reescale intensities and not visible effect will appear. For example we can use vmin = 0 and vmax = 50000

### Brightness

* We can increase/decrease the brightness of the image by just adding/substracting a constant value to the image intensity matrix(image data array).

* To test this we will create a new image data array equal to the used before but adding a constant value to all the values in the image data array.

```
image_data_bri = image_data + constant
```

* To check the results we will plot the resulting histogram with and a 2D slice of the image data array  side by side. 



### Contrast

* We can increase/decrease the contrast of an image by  multiplying/dividing the image by a constant, this widens/shortens the distance between intensity values in the image.

* To test this we will create a new image data array equal to the used before but multipliying by a constant value to all the values in the image data array.
image_data_contrast = image_data + constant
To check the results we will plot the resulting histogram with and a 2D slice of the image data array side by side.


### Histogram equalization

Histogram equalization is a method in image processing of contrast adjustment using the image's histogram. This method usually increases the global contrast of many images, especially when the image is represented by a narrow range of intensity values. Through this adjustment, the intensities can be better distributed on the histogram utilizing the full range of intensities evenly. This allows for areas of lower local contrast to gain a higher contrast. Histogram equalization accomplishes this by effectively spreading out the highly populated intensity values which use to degrade image contrast (source: [Wiki](https://en.wikipedia.org/wiki/Histogram_equalization)).

To test the histogram equalization in our sample image:
* We can use the *exposure.equalize_hist* function from *skimage* library

* Try to plot the original histogram and original image together with the histogram and resulting image  after equalization.

Why does it happens? 

Try to define a mask that solves this problem.

### Filtering




Convolution Filters (also known as kernels) are used with images for blurring, sharpening, embossing, edge detection, and more. This is accomplished by doing a convolution between a kernel and an image.

**Gaussian filter**: The Gaussian smoothing filter is a 2-D convolution filter that is used to `blur' images and remove detail and noise. 

  * Use the *filters.gaussian* command from skimage library to apply the filter on the image.

  * Plot the original and filtered images side-by-side and examine the differences.



**Laplacian filter:** The Laplacian is a 2-D isotropic measure of the 2nd spatial derivative of an image. The Laplacian of an image highlights regions of rapid intensity change and is therefore often used for edge detection. 

  * Use the *filters.laplace* command from skimage library to apply the filter on the image.

  * Plot the original and filtered images side-by-side and examine the differences.

**Custom filter:** By defining a custom kernel we can create our own filters to extract custom features.

  * Define your own filter by just defining a 3x3x3 kernel. 

  * Use the *filters.convolve* command from skimage library to apply the filter on the image.

  * Plot the original and filtered images side-by-side and examine the differences.

## 1.5 Affine transforms 

* Trasformation are crucial to understand and combine medical images
* Images (and volumes) can be thought of as fields (or functions) of space.
* Some examples are linear transforms are: translation, rotation or scaling 
* Rotation and scaling can be each written as matrix vector products
* We cannot do that for translations. But we can embed translations into a matrix-vector product if we bump up the dimensión by 1.

Normally we don't apply affine transformations directly on the image headers instead we use libraries or programs that do it for us. 
But in this case to exemplify what we have seen in the theory classes we can try to apply some basic transformations simply by modifying some parameters of the image header that define its position and orientation. 

To apply a affine transformation to an image we can extract the affine transformation that represents the position and orientation of the image contained in the nifty file, apply the desired transform and then save back to the image.
To do so we will do the following steps:

1.   Obtain the affine transform of the image "img.affine"
2.   Apply the desired transformations to the affine matrix
3.   Use the set_qform / set_sform methods of the original image "img" to include the new transformation affine to the image header.


### Translation

* As a first attempt we can try to move 10 mm in the x axis. 
* To do so we need to add 10 mm to the [0,3] position of the affine matrix.

### Rotation

* As a first attempt we can try to rotate the image 45º in a selected axis.
* How can we do that?

### Scaling

* As a first attemp we can try to enlarge the image two times the original.
* How can we do that?

# Part 2 -  Basic pipeline for neuroimaging
So far we have been studying and applying basic operations to understand the basics of image processing, but when we work on a day-to-day basis we use image processing libraries that allow us to define fast image processing and analysis flows. 

# Regarding to ANTsPy

Advanced Normalization Tools (ANTs) is useful for managing, interpreting and visualizing multidimensional data. ANTs is popularly considered a state-of-the-art medical image registration and segmentation toolkit. ANTsR is an emerging tool supporting standardized multimodality image analysis. ANTs depends on the Insight ToolKit (ITK), a widely used medical image processing library to which ANTs developers contribute.

ANTsPy is a Python library which wraps the C++ biomedical image processing library ANTs, matches much of the statistical capabilities of ANTsR, and allows seamless integration with numpy, scikit-learn, and the greater Python community.

Due ANTsPyx is not available by default at Google Colab so we need to install it using pip.

A basic pipeline for neuroimaging consist on the following steps:


1.   Loading the image
2.   Resample the images to set a common resolution for the whole dataset.
3.   Bias field correction: Bias field signal is a low-frequency and very smooth signal that corrupts MRI images specially those produced by old MRI (Magnetic Resonance Imaging) 
4.   Denoising: MR images are affected by random noise which limits the accuracy of any quantitative measurements from the data. To remove the noise from MRI is needed  to generate the MR images with high “signal-to-noise ratio”.
5.   Registration to a common space




### Install ANTsPyx
Due ANTsPyx is not available by default at Google Colab so we need to install it using !pip.


### Load the images

* Load the image we want to process. To do so we will process the *IVY_GAP_W01/T1c/T1.nii*
* Load the image we want to use as a reference for the common space. The reference image to be used will be *MNI_ATLAS/mni_icbm152_t1_tal_nlin_sym_09c.nii*. This image will be used in the "Register the image to a common reference space" step. 
* To do so we can use the command *ants.image_read*

### Resample the image we want to process
* To do so we can use the command ants.resample_image
* We can resample images to a resolution of 5x5x5 mm. This resolution is not optimal but it will make all the processing faster.
* Save the resulting images in nifti using the *ants.image_write*: command

### Perform a bias field correction

*  What does bieas field correction does? You can find a description on https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html 
*   To do so we can use the command *ants.n4_bias_field_correction*
*   We will use the default parameters.
*  Save the resulting images in nifti using the *ants.image_write*: command



### Remove noise from the images


*   To do so we can use the command *ants.denoise_image*
*   We will use the default parameters
*   Save the resulting images in nifti using the *ants.image_write*: command

### Register the image to a common reference space

*   To do so we will use the command *ants.registration*
*   We will register the image using two different types of transform "AffineFast" (affine transformation) and "SyN" (a non-linear trasformation) 


* Save the resulting images in nifti using the *ants.image_write*: command

* Once we have both images registered to a common space using linear and non-linear transformations we can compare them with the MNI image used as a reference in ITK-Snap